# Plotting time-correlated ZNE data

## Usage

Meant for plotting/saving images of data created by executing the [`run.ipynb`](./run.ipynb) notebook.

1. Run from Jupyter notebook / Colab.
2. Run with Papermill.
  * `papermill plot.ipynb out.ipynb -p param1 value1 -p param2 value2 ...`
  * *Tip*: Do `papermill plot.ipynb --help-notebook` to see all parameters.

## Imports

In [1]:
import glob
import os
import time

import numpy as np
import matplotlib.pyplot as plt

## Settings

In [2]:
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.size": 16,
        "font.weight": "bold",
    }
)
%matplotlib inline

## Constants

In [3]:
"""Filename and saving conventions."""
# Filename keys.
scale_factor_fnames_key = "scale_factors_*"
expectation_values_fnames_key = "all_expectation_values_*"

# These are the rows in expectation values filenames.
labels = (
    "True",     # First row is exact 𝐸(𝜆) curve.
    "Pulse",    # Second row is 𝐸(𝜆) curve for (approximate) pulse stretching.
    "Global",   # Third row is 𝐸(𝜆) curve for global folding.
    "Local",    # Fourth row is 𝐸(𝜆) curve for local folding.
    "Trotter",  # Fifth row is 𝐸(𝜆) curve for Trotter scaling.
)

## Runtime parameters

In [4]:
"""Parameters and default values. 

Note: Papermill inserts a new cell after this one if new parameter values are provided.
"""
# Data directory.
prefix: str = "./"

# Benchmark circuit parameters.
circuit_type: str = "rb"
depth: int = 10
nqubits: int = 1

# Noise parameters.
noise_type: str = "lowpass"
base_power: float = 0.01
cutoff_as_fraction_of_pi: float = 0.01  # Lowpass cutoff for lowpass noise.
alpha: float = 2.0  # The α in 1 / f^α noise.

# Scaling parameters.
local_fold_key: str = "random"
max_scale_factor: int = 9

# Plotting options.
ymin: float = -0.01
ymax: float = 0.40
fill_below = False
fill_std = False

# Option to save plots.
save: bool = False
save_type: str = "pdf"

# Other miscellaneous parameters.
num_monte_carlo: int = 1000
verbosity: int = 1

## Load data

In [5]:
noise_params = {
    "lowpass": f"cutoff_{cutoff_as_fraction_of_pi}",
    "pink": "alpha_%0.2f" %alpha,
}.get(noise_type)
dir_name = f"tzne_circuit_type_{circuit_type}_nqubits_{nqubits}_depth_{depth}_noise_type_{noise_type}_base_power_{base_power}_{noise_params}_max_scale_factor_{max_scale_factor}_num_monte_carlo_{num_monte_carlo}"

exp_values_fnames = glob.glob(os.path.join(prefix, dir_name, expectation_values_fnames_key))

if len(exp_values_fnames) == 0:
    found_dirs = "\n".join(
        [s for s in glob.glob(os.path.join(prefix, '*')) if os.path.isdir(s)]
    )
    raise ValueError(
        f"Did not find any directories in {prefix} matching the pattern "
        f"{dir_name}. The directories in {prefix} are \n{found_dirs}"
    )


print(f"Found {len(exp_values_fnames)} matching filenames.")
all_expectation_values = np.array([
    np.loadtxt(fname) for fname in exp_values_fnames
])

ValueError: Did not find any directories in ./ matching the pattern tzne_circuit_type_rb_nqubits_1_depth_10_noise_type_lowpass_base_power_0.01_cutoff_0.01_max_scale_factor_9_num_monte_carlo_1000. The directories in ./ are 


## Plot the average $E(\lambda)$ curves for all scaling methods

In [ ]:
#@title Plotting code.
fig = plt.figure(figsize=(7, 4))
colors = ("tab:blue", "tab:orange", "tab:green", "tab:red", "teal")

avgs = np.average(all_expectation_values, axis=0)
stds = np.std(all_expectation_values, axis=0)

for i, (avg, std) in enumerate(zip(avgs, stds)):
    scale_factors = range(1, 2 * len(avg) + 1, 2)
    plt.errorbar(
        x=scale_factors,
        y=avg,
        yerr=std,
        ls="dashdot" if i != 0 else "-",
        capsize=7,
        lw=2.5,
        color=colors[i],
        label=labels[i],
    )
    plt.fill_between(x=scale_factors, y1=avg - std, y2=avg + std, alpha=0.2)

# Style.
plt.xticks(scale_factors)
plt.xlabel(r"$\lambda$")
plt.ylabel(r"$E(\lambda)$")
plt.legend()
plt.tight_layout();

if save:
    plt.savefig(os.path.join(os.path.join(prefix, dir_name), "000_expvals" + "." + save_type))

## Plot the relative errors in $E(\lambda)$ curves

In [ ]:
fig = plt.figure(figsize=(7, 4))

true = avgs[0]
for i, (avg, std) in enumerate(zip(avgs[1:], stds[1:])):
    i += 1
    scale_factors = range(1, 2 * len(avg) + 1, 2)
    error = np.abs((true - avg) / true)
    plt.plot(
        scale_factors, error, "--s", lw=2, alpha=0.85, ms=8, color=colors[i], mec="black", label=labels[i]
    )
    if fill_below:   
        plt.fill_between(x=scale_factors, y1=error, alpha=0.2, color=colors[i])
    if fill_std:
        plt.fill_between(x=scale_factors, y1=error + std, y2=error - std, alpha=0.2, color=colors[i])

# Style.
plt.xticks(scale_factors)
plt.xlabel(r"$\lambda$")
plt.ylabel(r"$\left| \frac{ E(\lambda) - E^*(\lambda) }{ E^*(\lambda) } \right|$")
plt.legend()
plt.tight_layout();
plt.ylim(ymin, ymax)

if save:
    plt.savefig(os.path.join(os.path.join(prefix, dir_name), "000_errors" + "." + save_type))